### Пример предобработки и векторизации текстовых данных с помощью word2vec

#### Инициализация spacy и загрузка модели для русского языка

In [2]:
import spacy

sp = spacy.load("ru_core_news_lg")

#### Загрузка содержимого текстовых документов

In [3]:
import pandas as pd
from docx import Document
import os


def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return "\n".join(full_text)


def load_docs(dataset_path):
    df = pd.DataFrame(columns=["doc", "text"])
    for file_path in os.listdir(dataset_path):
        if file_path.startswith("~$"):
            continue
        text = read_docx(dataset_path + file_path)
        df.loc[len(df.index)] = [file_path, text]
    return df


df = load_docs("data/text/")
df["type"] = df.apply(lambda row: 0 if str(row["doc"]).startswith("tz_") else 1, axis=1)
df.info()
df.sort_values(by=["doc"], inplace=True)

display(df.head(), df.tail())

<class 'pandas.core.frame.DataFrame'>
Index: 41 entries, 0 to 40
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   doc     41 non-null     object
 1   text    41 non-null     object
 2   type    41 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


,doc,text,type
16,tz_01.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0
19,tz_02.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0
28,tz_03.docx,2.2. Техническое задание\nОбщие сведения:\nВ д...,0
35,tz_04.docx,Техническое задание\n2.2.1 Общие сведения\nИнт...,0
38,tz_05.docx,2.2 Техническое задание\n2.2.1 Общие сведения....,0


,doc,text,type
25,Этапы разработки проекта2.docx,Этапы разработки проекта: заключительные стади...,1
21,Этапы разработки проекта3.docx,Этапы разработки проекта: определение стратеги...,1
40,Этапы разработки проекта4.docx,"Этапы разработки проекта: реализация, тестиров...",1
30,Этапы разработки проекта5.docx,Этапы разработки проекта: стратегия и анализ\n...,1
22,Язык манипуляции данными.docx,2.1.3. Язык манипуляции данными (ЯМД)\nЯзык ма...,1


#### Предобработка текстовых данных
- приведение к нижнему регистру;
- удаление стоп-слов;
- удаление знаков препинания и спец-символов;
- лемматизация;
- формирование би-грамм с помощью Phraser, Phrases.

У Phrases можно изменять параметры: min_count и threshold (минимальное количество вхождений и порог оценки устойчивости) для повышения качества полученных би-грамм.

In [4]:
from gensim.models.phrases import Phraser, Phrases

def prep_text(text):
    doc = sp(text)
    lower_sents = []
    for sent in doc.sents:
        lower_sents.append([word.lemma_.lower() for word in sent if not word.is_punct and not word.is_stop and not word.is_space])
    lower_bigram = Phraser(Phrases(lower_sents))
    clean_sents = []
    for sent in lower_sents:
        clean_sents.append(lower_bigram[sent])
    return clean_sents

df["prep_text"] = df.apply(lambda row: prep_text(row["text"]), axis=1)
df

,doc,text,type,prep_text
16,tz_01.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0,"[[2.2, технический, задание, 2.2.1, общий, све..."
19,tz_02.docx,2.2 Техническое задание\n2.2.1 Общие сведения\...,0,"[[2.2, технический, задание, 2.2.1, общий, све..."
28,tz_03.docx,2.2. Техническое задание\nОбщие сведения:\nВ д...,0,"[[2.2], [технический, задание, общий, сведение..."
35,tz_04.docx,Техническое задание\n2.2.1 Общие сведения\nИнт...,0,"[[технический, задание, 2.2.1, общий, сведение..."
38,tz_05.docx,2.2 Техническое задание\n2.2.1 Общие сведения....,0,"[[2.2, технический, задание, 2.2.1, общий, све..."
2,tz_06.docx,2.2 Техническое задание\t\n1.Общие сведения\nП...,0,"[[2.2, технический, задание, 1.общие, сведение..."
4,tz_07.docx,Техническое задание\nОбщие сведения\nВ данном ...,0,"[[технический, задание, общий, сведение, разде..."
33,tz_08.docx,Техническое задание\n1 Общие сведения\n1.1 Пол...,0,"[[технический, задание, 1, общий, сведение, 1...."
39,tz_09.docx,2.2. Техническое задание\n2.2.1.\n\nОбщие свед...,0,"[[2.2], [технический, задание, 2.2.1, общий, с..."
1,tz_10.docx,2.2. Техническое задание\n2.2.1. Общие сведени...,0,"[[2.2], [технический, задание, 2.2.1], [общий,..."


#### Векторизация текстовых данных с помощью word2vec
- функция explode объединяет все массивы столбца prep_text в один;
- будет получено 64-мерное пространство (векторы длиной 64 элемента);
- используется метод skip-gram (sg=1). Данный метод предпочтительно использовать на небольшом корпусе, а CBOW на большом (sg=0);
- размер окна 10 (по 10 слов слева и справа от текущего для вычисления контекста);
- min_count - минимальное количество вхождений слова в корпус для включения в n-мерное пространство;
- workers - количество потоков для вычисления;
- seed - задание начального состояния для повторяемости (random_state).


In [5]:
from gensim.models.word2vec import Word2Vec

word2vec = Word2Vec(
    sentences=df["prep_text"].explode().tolist(),
    vector_size=64,
    sg=1,
    window=10,
    epochs=5,
    min_count=10,
    workers=4,
    seed=9,
)

#### Вывод уникальных идентификаторов для каждого слова

Слова в пространстве хранятся в виде числового идентификатора для сокращения потребления памяти

In [6]:
word2vec.wv.key_to_index

{'система': 0,
 'работа': 1,
 'требование': 2,
 'база': 3,
 'пользователь': 4,
 'разработка': 5,
 'модель': 6,
 'информация': 7,
 'субд': 8,
 'этап': 9,
 'ошибка': 10,
 'функция': 11,
 'являться': 12,
 'таблица': 13,
 'средство': 14,
 'проект': 15,
 'сервер': 16,
 'процесс': 17,
 'документ': 18,
 'программа': 19,
 'использовать': 20,
 'состояние': 21,
 'проектирование': 22,
 'программный': 23,
 'случай': 24,
 'создание': 25,
 'модуль': 26,
 'время': 27,
 'объект': 28,
 'заказчик': 29,
 'приложение': 30,
 'реализация': 31,
 'правило': 32,
 'результат': 33,
 'вид': 34,
 'разработчик': 35,
 'возможность': 36,
 'технический': 37,
 'метод': 38,
 'тестирование': 39,
 'следующий': 40,
 'задача': 41,
 'использование': 42,
 'обработка': 43,
 'часть': 44,
 'позволять': 45,
 'клиент': 46,
 'интерфейс': 47,
 'переход': 48,
 'выполнение': 49,
 'обеспечение': 50,
 'информационный_система': 51,
 'новый': 52,
 'изменение': 53,
 'атрибут': 54,
 'основный': 55,
 'код': 56,
 'решение': 57,
 'запрос': 58,

#### Вывод трех наиболее похожих по смыслу слова для слова 1С

In [7]:
word2vec.wv.most_similar("1с", topn=3)

[('8.1', 0.9879903197288513),
 ('бухгалтерия', 0.980316698551178),
 ('навык', 0.9310787320137024)]

#### Координаты слова 1С в 64-мерном пространстве

In [8]:
word2vec.wv["1с"]

array([-5.72758615e-01, -4.64142323e-01, -1.41238064e-01, -6.03196502e-01,
       -3.06068182e-01,  1.47344675e-02, -8.86413306e-02,  6.53996249e-05,
        2.22017527e-01, -1.03922300e-01,  4.06073749e-01, -5.07203415e-02,
        1.23037435e-01, -1.68328971e-01, -8.36593136e-02,  2.31885687e-01,
       -1.16688296e-01,  1.43522806e-02, -5.78690231e-01,  3.08529168e-01,
        3.75707522e-02,  1.13559164e-01,  3.83746147e-01,  2.44788274e-01,
       -5.16302809e-02, -8.68294463e-02,  7.44756520e-01,  2.69289196e-01,
       -1.38882861e-01,  3.19430903e-02,  2.82276541e-01, -1.97692990e-01,
       -1.43509433e-01, -1.92271873e-01, -1.49369061e-01,  6.34338081e-01,
        1.60252139e-01,  6.62923336e-01,  9.45296362e-02, -1.62105456e-01,
       -2.70663768e-01,  2.80816615e-01,  4.22135949e-01, -1.53156146e-01,
        1.51129171e-01,  3.92942369e-01, -5.38962841e-01, -1.52189329e-01,
       -7.02899694e-01, -4.62726772e-01, -8.09689939e-01,  2.60302156e-01,
        2.99924672e-01,  

#### Поиск лишнего по смыслу слова (самое отдаленное по смыслу слово)

In [9]:
word2vec.wv.doesnt_match("java php 1с oracle заказчик".split())

'заказчик'

#### Смысловое расстояние между словами (расстояние в пространстве)

In [10]:
word2vec.wv.similarity("java", "javascript")

0.83737653

In [11]:
word2vec.wv.similarity("java", "1с")

0.7350806

#### Векторная арифметика: 1c - бухгалтерия + машина =

In [12]:
word2vec.wv.most_similar(positive=["1с", "машина"], negative=["бухгалтерия"])

[('лингвистический', 0.9782435297966003),
 ('php', 0.9754980802536011),
 ('mysql', 0.9643309712409973),
 ('java', 0.963462769985199),
 ('знание', 0.950555682182312),
 ('apache', 0.9452742338180542),
 ('1с_предприятие', 0.943841278553009),
 ('windows_nt', 0.9398906826972961),
 ('работать', 0.9393702745437622),
 ('базовый', 0.9373704791069031)]

#### Векторная арифметика: субд - база + язык =

In [13]:
word2vec.wv.most_similar(positive=["субд", "язык"], negative=["база"])

[('программирование', 0.8422903418540955),
 ('java', 0.7840348482131958),
 ('использовать', 0.7770466804504395),
 ('php', 0.7726089954376221),
 ('sql', 0.7726024389266968),
 ('лингвистический', 0.7516488432884216),
 ('производитель', 0.7262539267539978),
 ('написание', 0.7253801226615906),
 ('использование', 0.7136848568916321),
 ('мощный', 0.7123106718063354)]

#### Векторная арифметика: java - машина + бухгалтерия =

In [14]:
word2vec.wv.most_similar(positive=["java", "бухгалтерия"], negative=["машина"])

[('1с', 0.9639856815338135),
 ('среда', 0.942729651927948),
 ('8.1', 0.9425337910652161),
 ('ориентировать', 0.9270872473716736),
 ('web', 0.915542483329773),
 ('платформа', 0.9038816094398499),
 ('навык', 0.8979926705360413),
 ('программный', 0.8968376517295837),
 ('компания', 0.8852423429489136),
 ('лингвистический', 0.8833838105201721)]

#### Векторная арифметика: 866 - мгц + мбайт =

In [15]:
word2vec.wv.most_similar(positive=["866", "мбайт"], negative=["мгц"])

[('256', 0.9967159628868103),
 ('512', 0.9953411221504211),
 ('pentium', 0.9951350688934326),
 ('iii', 0.9946882724761963),
 ('celeron', 0.9945207834243774),
 ('iv', 0.9916473031044006),
 ('intel', 0.9878745079040527),
 ('2003', 0.9863777756690979),
 ('процессор', 0.9855839610099792),
 ('c', 0.9850897192955017)]

#### Векторная арифметика: 866 - процессор + память =

In [18]:
word2vec.wv.most_similar(positive=["866", "память"], negative=["процессор"])

[('256', 0.9890534281730652),
 ('мбайт', 0.9886034727096558),
 ('мгц', 0.9871484637260437),
 ('pentium', 0.9858012199401855),
 ('celeron', 0.9855312705039978),
 ('c', 0.9849308133125305),
 ('512', 0.9843060970306396),
 ('iii', 0.9841062426567078),
 ('iv', 0.9838903546333313),
 ('высоко', 0.9828835129737854)]

#### Понижение размерности векторного пространства с 64 до 2 измерений

In [19]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, max_iter=1000, random_state=9)
coords_df = pd.DataFrame(
    tsne.fit_transform(word2vec.wv[word2vec.wv.key_to_index]), columns=["x", "y"]
)
coords_df["token"] = word2vec.wv.key_to_index.keys()
coords_df

,x,y,token
0,-24.734394,-7.651334,система
1,-30.030426,-18.815897,работа
2,-26.821131,2.513596,требование
3,-8.133460,24.581547,база
4,-20.158037,3.450009,пользователь
...,...,...,...
1038,6.665592,-2.917084,некий
1039,18.308809,9.314460,самым
1040,12.663247,-1.679625,похожий
1041,7.762640,1.464980,целесообразный


#### Визуализация векторного пространства

Диаграмма интерактивная и позволяет изменять масштаб и перемещаться

In [20]:
from bokeh.io import output_notebook
from bokeh.plotting import show, figure

output_notebook()
p = figure(width=800, height=800)
p.text(x=coords_df.x, y=coords_df.y, text=coords_df.token) # type: ignore
show(p)

Loading BokehJS ...